# Music Data Modeling Project
In this project, we apply Data Modeling with Cassandra and build an ETL pipeline using Python. We will build a Data Model around our queries that we want to get answers for. For our use case we want below answers:
1. Get details of a song that was herad on the music app history during a particular session.
2. Get songs played by a user during particular session on music app.
3. Get all users from the music app history who listened to a particular song.

# Part I. ETL Pipeline for Pre-Processing the Files

In [4]:
# import python packages
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [5]:
# process original event csv data files
# check current working directory
print(f"current working directory: {os.getcwd()}")
# get directory of the data files
filepath = os.getcwd() + '/event_data'
# create a list of the files and collect each filepath
for root,dirs,files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    filepath_list = glob.glob(os.path.join(root,'*'))
    print(filepath_list)

current working directory: /Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_Data_Modeling
['/Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_Data_Modeling/event_data/2018-11-15-events.csv', '/Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_Data_Modeling/event_data/2018-11-22-events.csv', '/Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_Data_Modeling/event_data/2018-11-09-events.csv', '/Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_Data_Modeling/event_data/2018-11-18-events.csv', '/Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_Data_Modeling/event_data/2018-11-04-events.csv', '/Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_Data_Modeling/event_data/2018-11-01-events.csv', '/Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_D

# Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [12]:
full_data_rows_list = []

for fp in filepath_list:
    with open(fp,'r',encoding='utf8',newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        for l in csvreader:
            full_data_rows_list.append(l)

print(f'total rows: {len(full_data_rows_list)}')
print(f'sample data:\n {full_data_rows_list[:5]}')

csv.register_dialect('Dialect',quoting=csv.QUOTE_ALL,skipinitialspace=True)

with open('event_datafile.csv','w',encoding='utf8',newline='') as f:
    writer = csv.writer(f, dialect='Dialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName',
                     'length','level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if(row[0]==''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))



total rows: 8056
sample data:
 [['Harmonia', 'Logged In', 'Ryan', 'M', '0', 'Smith', '655.77751', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'Sehr kosmisch', '200', '1.54224E+12', '26'], ['The Prodigy', 'Logged In', 'Ryan', 'M', '1', 'Smith', '260.07465', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'The Big Gundown', '200', '1.54224E+12', '26'], ['Train', 'Logged In', 'Ryan', 'M', '2', 'Smith', '205.45261', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'Marry Me', '200', '1.54224E+12', '26'], ['', 'Logged In', 'Wyatt', 'M', '0', 'Scott', '', 'free', 'Eureka-Arcata-Fortuna, CA', 'GET', 'Home', '1.54087E+12', '563', '', '200', '1.54225E+12', '9'], ['', 'Logged In', 'Austin', 'M', '0', 'Rosales', '', 'free', 'New York-Newark-Jersey City, NY-NJ-PA', 'GET', 'Home', '1.54106E+12', '521', '', '200', '1.54225E+12', '12']]


In [13]:
with open('event_datafile.csv','r',encoding='utf8') as f:
    print(sum(1 for line in f))

6821


Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### Creating a Cluster

In [2]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("Connection Established !!")
except Exception as e:
    print(f"Connection Failed !! Error : {e}")

Connection Established !!


#### Creating Keyspace

In [3]:
keyspace_query = """CREATE KEYSPACE IF NOT EXISTS sparkify 
                    with REPLICATION = 
                    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                """

# Creating Keyspace
try:
    session.execute(keyspace_query)
except Exception as e:
    print(f"Failed to create keyspace!! Error : {e}")

In [4]:
# Setting KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

Data Model requirement

1. Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [5]:
# Creating table for query1 
create_query1 = """CREATE TABLE IF NOT EXISTS session_item (artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"""

try: 
    session.execute(create_query1)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

Table Created!!


In [7]:
import csv
# Using the event file
file = 'event_datafile.csv'

# Reading csv file and inserting rows into cassandra tables.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_item (artist, song, length, sessionId, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s) "
        session.execute(query, (line[0], line[10], float(line[5]), int(line[8]), int(line[3])) )

In [8]:
# SELECT statement to verify the data was entered into the table
select_query1 = "SELECT artist, song, length FROM  session_item where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(select_query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song='50', length=495.30731201171875)


2. Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [9]:
# Creating table for query2 
create_query2 = """CREATE TABLE IF NOT EXISTS user_session (sessionId int, userId int, artist text, song text, firstName text, lastName text, itemInSession int, PRIMARY KEY ((sessionId, userId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC) """

try: 
    session.execute(create_query2)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

Table Created!!


In [10]:
file = 'event_datafile.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (sessionId, userId, artist, song, firstName, lastName, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s) "
        session.execute(query, (int(line[8]), int(line[10]), line[0], line[9], line[1], line[4], int(line[3])  ) )

In [11]:
# SELECT statement to verify the data was entered into the table
select_query2 = "SELECT artist, song, firstName, lastName FROM  user_session where sessionId = 182 and userId = 10"
try:
    rows = session.execute(select_query2)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [12]:
# Creating table for query3

create_query3 = """CREATE TABLE IF NOT EXISTS user_song (song text, userId int, firstName text, lastName text, PRIMARY KEY ((song), userId))"""

try: 
    session.execute(create_query3)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

Table Created!!


In [13]:
file = 'event_datafile.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (song, userId, firstName, lastName) "
        query = query + " VALUES (%s, %s, %s, %s) "
        session.execute(query, (  line[9], int(line[10]), line[1], line[4] )  )

In [14]:
# SELECT statement to verify the data was entered into the table
select_query2 = "SELECT song, firstName, lastName FROM user_song where song = 'All Hands Against His Own'"
try:
    rows = session.execute(select_query2)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [15]:
session.execute("DROP TABLE IF EXISTS sparkify.session_item")
session.execute("DROP TABLE IF EXISTS sparkify.user_session")
session.execute("DROP TABLE IF EXISTS sparkify.user_song")

### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()